In [4]:
# Ferramenta VaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Requisições a webservices
import requests

In [5]:
def translate_row_text(row):
        
    text = row['full_text']
    
    from_lang = row['lang']
    to_lang = "en"
    
    # Verificando se o texto do tweet já está escrito na língua inglesa
    if (from_lang == "en") or (from_lang == "en-US"):
        translation = row['full_text']
        
    else:  
        api_url = "http://mymemory.translated.net/api/get?de={}&q={}&langpair={}|{}".format('mariogmp@gmail.com',text, from_lang, to_lang)
        hdrs = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                'Accept-Encoding': 'none',
                'Accept-Language': 'en-US,en;q=0.8',
                'Connection': 'keep-alive'}
        response = requests.get(api_url, headers=hdrs)
        response_json = json.loads(response.text)
        translation = response_json["responseData"]["translatedText"]

    return translation

In [6]:
def calculate_scores(tweets):

    for tweet in tweets:
                
        # Traduzindo o texto para o inglês
        translated_text = translate_row_text(row)
        translated_text = tweet['text']  
        
        # Efetuando o cálculo do score de sentimento para o texto
        analyzer = SentimentIntensityAnalyzer()
        vs = analyzer.polarity_scores(translated_text)
        
        # Recuperando o score e adicionando em uma nova célula do dataframe
        collection.update_one({"tweet_id": tweet['tweet_id']}, {'$set':{"score": vs['compound']}})      